## Change Score Models with more than one covariate.

### 1. Introduction
As explained during the theoretical part of this seminar, one of the advantages of Change Score Models over MLMs is the high flexibility of the SEM framework when incorporating multiple covariates. Do you remember that we also wanted to assess whether EM had an effect on the depression change? 

In [1]:
#######################################################
## Specify R environment for rpy2
import os
os.environ['R_HOME'] = r'C:/Program Files/R/R-4.4.3'  # Replace with your R path
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from rpy2.robjects import Formula
import contextlib
# Ipython extension for plotting
%load_ext rpy2.ipython
########################################################

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display


depression = pd.read_csv("../MLM_and_CSM/Datasets/depression.csv")
depression.head()


c:\Users\danie\Documents\envPython\psy112\lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(


,id,group,pre,post,EM,change
0,1,0,34,76,25,42
1,2,0,87,51,34,-36
2,3,0,68,71,15,3
3,4,0,38,72,41,34
4,5,0,82,47,14,-35


### 2. Modeling (Exercise)

Use the next code chunk to extend the previous "m2" model to include a second predictor ("EM") of change and pre-therapy scores. 

In [2]:
# Activate pandas2ri for automatic conversion
pandas2ri.activate()

# Import the lavaan package in R
ro.r('library(lavaan)')

# Import semPlot for plotting SEM paths
ro.r('library(semPlot)')

# Load the dataset into R
ro.globalenv['depression'] = pandas2ri.py2rpy(depression)

# Define the SEM model
ro.r('''
m3 <-  '

# Fixing change score loading to 1
ph_change =~ 1*post

#Fixing baseline loadidng to 1
post ~ 1*pre

#Fixing post-treatment score residual variance to 0
post ~~ 0*post

#Fixing post-treatment score intercept to 0
post ~ 0 * 1 

# Freely estimate phantom variable and baseline means. The default would fix them to 0
ph_change ~ 1 
pre ~ 1

# Include covariance between phantom change variable and baseline
ph_change ~~ pre

# Adding group as covariate
ph_change ~ group
pre ~ group

# Adding EM as covariate
ph_change ~ EM
pre ~ EM

'
''')

# Fit the SEM model
ro.r('fit3 <- sem(m3, data=depression, meanstructure=TRUE)')

# Display the summary of the SEM model
summary = ro.r('summary(fit3, fit.measures=TRUE, standardized=TRUE)')
print(summary)


R[write to console]: This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.



lavaan 0.6-19 ended normally after 226 iterations

  Estimator                                         ML
  Optimization method                           NLMINB
  Number of model parameters                         9

  Number of observations                           100

Model Test User Model:
                                                      
  Test statistic                                 0.000
  Degrees of freedom                                 0

Model Test Baseline Model:

  Test statistic                                60.857
  Degrees of freedom                                 5
  P-value                                        0.000

User Model versus Baseline Model:

  Comparative Fit Index (CFI)                    1.000
  Tucker-Lewis Index (TLI)                       1.000

Loglikelihood and Information Criteria:

  Loglikelihood user model (H0)               -832.579
  Loglikelihood unrestricted model (H1)       -832.579
                                               

### 3. Interpretation

Interpretation follows the same logic as in the previous model. Note that neither on change, nor on pre-therapy scores, a significant effect of EM was found (p = 0.479 and p = 0.621, respectively).

Note that in this case CSMs and MLMs allowed us to answer the same questions. The selection of either will depend on several factors but certainly, the best is to know both! Especially because for complex evaluation designs the two must be combined - think for example about clustering of patients into therapists. If we wanted to address the same questions as before, but additionally investigate whether change in the intervention group depend on the therapist, we need a Multilevel Change Score Model. Don't worry, we do not do that here. Such a model is even difficult for lavaan at the moment. People use a commercial software called Mplus for estimating such models. Note, that these type of models are very prominent in the clinical literature and evaluation practice. 